# Live API demo.

There is a live demo API service hosted on Render. You can see the Swagger docs for the demo API at the following address: https://queens-5stl.onrender.com/docs. These docs apply to the "real" API as well.

The Render demo deploys the API based on a sample database, which has been obtained ingesting all tables published in the 2025 edition of DUKES. The demo DB will be update periodically to showcase new features or new available endpoints. 

Please note that the demo is hosted under a free plan with limited computing power. Requests (especially on startup) might take longer than expected. 

The examples below demonstrate the use of the metadata and data endpoints, filtering and pagination. They **do not require `queens` to be installed**.

In [5]:
import json
import requests
import pandas as pd

BASE = "https://queens-5stl.onrender.com"   # Render URL
COLLECTION = "dukes"

# first request might take a while, subsequent requests will be almost immediate.
TIMEOUT = 90

## Meatadata endpoint

Returns the queryable columns for each table, the number of non-nulls, the data type and the unique values for each column.

In [6]:
# base endpoint
meta_url = f"{BASE}/metadata/{COLLECTION}"

# get response
response = requests.get(meta_url, params={"table_name": "J.1"}, timeout=TIMEOUT)
response.raise_for_status()

# collect the data and transform into a dataframe
payload = response.json()
meta_df = pd.DataFrame(payload["data"])

meta_df.head()


,data_collection,table_name,column_name,n_non_nulls,n_unique,dtype
0,dukes,J.1,category,12626,26,TEXT
1,dukes,J.1,fuel,12626,10,TEXT
2,dukes,J.1,group,12626,3,TEXT
3,dukes,J.1,item,12626,52,TEXT
4,dukes,J.1,label,12626,52,TEXT


## Data endpoint

Query the actual data from a data collection. Allows one `table_name` per request and supports complex filtering. See the documentation for filtering rules.

For large tables, pagination is implemented, with a default page limit of 1,000 rows. The user can request the various pages using the `next_cursor` value provided with the response.

The filters below are equivalent to the following SQL query:
```
SELECT *
FROM dukes_prod
WHERE
    year > 2010
    AND subgroup = 'Transformation'
    AND (fuel = 'Solid waste' OR fuel LIKE '%oil%')
```

In [17]:
data_url = f"{BASE}/data/{COLLECTION}"

filters = {
    "year": {"gte": 2010},
    "subgroup": "Transformation",
     "$or": [
        {"fuel": {"like": "%oil%"}},
        {"fuel": "Solid waste"}
    ]
   
}

params = {
    "table_name": "J.1",
    "limit": 1000,  # <= 5000
    "filters": json.dumps(filters)
}

r = requests.get(data_url, params=params, timeout=TIMEOUT)
r.raise_for_status()
payload = r.json()

table_descr = payload["table_description"]
page_df = pd.DataFrame(payload["data"])
next_cursor = payload["next_cursor"]

print(table_descr, page_df.shape, next_cursor)
page_df.head()

Heat sold reallocation, 1999 to 2024 (DUKES J.1) (396, 11) None


,table_name,row,label,year,group,subgroup,category,item,fuel,unit,value
0,J.1,9,Transformation,2024,Demand,Transformation,Transformation,Transformation,Fuel oil,ktonnes,-30.45
1,J.1,10,Electricity generation,2024,Demand,Transformation,Electricity generation,Electricity generation,Fuel oil,ktonnes,0.00
2,J.1,11,Major power producers,2024,Demand,Transformation,Electricity generation,Major power producers,Fuel oil,ktonnes,0.00
3,J.1,12,Autogenerators,2024,Demand,Transformation,Electricity generation,Autogenerators,Fuel oil,ktonnes,0.00
4,J.1,13,Heat generation,2024,Demand,Transformation,Heat generation,All plants,Fuel oil,ktonnes,-30.45


## Requesting the whole dataset

If a table has more than 5,000 rows (the max request limit), it is possible to request the whole dataset by looping multiple requests. Below is a small helper that implements this logic.

In [19]:
def fetch_all_pages(table_name: str, filters: dict = None, page_size: int = 5000) -> pd.DataFrame:
    
    url = f"{BASE}/data/{COLLECTION}"
    cursor = None
    frames = []

    while True:
        params = {"table_name": table_name, "limit": page_size}
        if filters:
            params["filters"] = json.dumps(filters)
        if cursor is not None:
            params["cursor"] = cursor

        r = requests.get(url, params=params, timeout=TIMEOUT)
        r.raise_for_status()
        j = r.json()

        frames.append(pd.DataFrame(j["data"]))
        cursor = j["next_cursor"]
        
        # next cursor will be eventually None when no more pages exist
        if cursor is None:
            break

    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

full_df = fetch_all_pages("J.1")

print(full_df.shape) 
full_df.head()


(12626, 11)


,table_name,row,label,year,group,subgroup,category,item,fuel,unit,value
0,J.1,0,Indigenous production,2024,Supply,Production,Production,Production,Coal,ktonnes,0.0
1,J.1,1,Imports,2024,Supply,Imports,Imports,Imports,Coal,ktonnes,0.0
2,J.1,2,Exports,2024,Supply,Exports,Exports,Exports,Coal,ktonnes,0.0
3,J.1,3,Marine bunkers,2024,Supply,Marine bunkers,Marine bunkers,Marine bunkers,Coal,ktonnes,0.0
4,J.1,4,Stock change,2024,Supply,Stock change,Stock change,Stock change,Coal,ktonnes,0.0
